In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.optimize import least_squares
import math
import pandas as pd
import csv
import datetime
from scipy import stats
from scipy import optimize
from datetime import date, timedelta
import random
import requests
from scipy.stats import truncnorm
import datetime

import statsmodels as sm
#import statsmodels.api as sm

from PIL import Image
from scipy.stats import norm

# Read data

## Cases

In [ ]:
df = pd.read_csv('/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/data/covid.csv')

## Population

In [ ]:
pop = pd.read_csv('/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/results/seiir_bootstrap/populacao_municipio.csv')

## Mobility data

In [ ]:
mob = pd.read_csv('/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/data/distancing_series.csv')

## Stringency

In [ ]:
strg = pd.read_csv('/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/data/stringency_series.csv')

# Create dataframe to analyse the following time series:

    * Daily number of cases
    * Acumulated number of cases
    * SMRI
    * Stringency
    * Rt

In [ ]:
df = df[df.date <= '2020-05-22']

In [ ]:
df.head()

In [ ]:
mob

In [ ]:
mob = mob[(mob.date >= '27-Feb-2020 00:00:00') & (mob.date <= '2020-05-22')]

In [ ]:
mob.head()

In [ ]:
series_state = df.groupby(['state', 'date'])['new_confirmed'].sum().reset_index()

In [ ]:
series_state.head()

In [ ]:
for value in mob.sig_uf:
    s = value
    
    print(s)
    
    cases = series_state[series_state.state == s]
    
    cases['cumulative'] = cases.new_confirmed.cumsum()
    
    mobs = mob[mob.sig_uf == s][(mob[mob.sig_uf == s].dt >= cases.iloc[0,1])]
    
    r, p = stats.pearsonr(cases.dropna()['new_confirmed']/sum(cases.dropna()['new_confirmed']), mobs.dropna()['isolated'])
    print(f"Scipy computed Pearson r: {r} and p-value: {p}")
    
    cases['isolated'] = np.array(mobs.isolated)
    
    casesLog = np.log(cases.filter(['new_confirmed','cumulative','isolated']))
    casesLog = casesLog.replace([np.inf, -np.inf], np.nan).fillna(0)
    
    cases['cumLog'] = casesLog.cumulative
    
    import functools
    cases.filter(['isolated','cumLog']).apply(functools.partial(test_stationarity))
    
    plt.scatter(cases.cumLog,cases.isolated, marker='o')
    
    model = VAR(cases.filter(['isolated','cumLog']))
    
    model1 = model.select_order()
    print(model1.summary())
    
    xcov = [crosscorr(cases.isolated, cases.cumLog, lag=i) for i in range(20)]
    print(xcov)
    
    x= cases.isolated 
    y= cases.cumLog

    n =len(x)
    l1 = (-1/n) + (2/math.sqrt(n))

    fig, ax1 = plt.subplots(1, 1, sharex=True)
    ax1.xcorr(x, y, usevlines=True, maxlags=None, normed=True, lw=2)

    ax1.axhline(l1, color='blue', linestyle='dashed', lw=1)

    ax1.grid(True)

    #ax2.acorr(x, usevlines=True, normed=True, maxlags=None, lw=2)
    #ax2.grid(True)


    plt.show()

In [ ]:
import datetime
days = 90
datetime.datetime(2020, 1, 1) + datetime.timedelta(days - 1)

In [ ]:
days = 25
datetime.datetime(2020, 3, 6) + datetime.timedelta(days - 1)

In [ ]:
cases.head()

In [ ]:
cases['cumulative'] = cases.new_confirmed.cumsum()

In [ ]:
mobs = mob[mob.sig_uf == s][(mob[mob.sig_uf == s].dt >= cases.iloc[0,1])]

In [ ]:
mobs.head(1)

In [ ]:
r, p = stats.pearsonr(cases.dropna()['new_confirmed']/sum(cases.dropna()['new_confirmed']), mobs.dropna()['isolated'])
print(f"Scipy computed Pearson r: {r} and p-value: {p}")

In [ ]:
cases['isolated'] = np.array(mobs.isolated)

In [ ]:
cases

In [ ]:
casesLog = np.log(cases.filter(['new_confirmed','cumulative','isolated']))
casesLog = casesLog.replace([np.inf, -np.inf], np.nan).fillna(0)

In [ ]:
casesLog.head(1)

In [ ]:
cases['cumLog'] = casesLog.cumulative

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [ ]:
import functools
cases.filter(['isolated','cumLog']).apply(functools.partial(test_stationarity))

In [ ]:
plt.scatter(cases.cumLog,cases.isolated, marker='o')

In [ ]:
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(cases.filter(['isolated','cumLog']))

In [ ]:
# Best model order (automatic selection)
model1 = model.select_order()
model1.summary()

In [ ]:
def crosscorr(datax, datay, lag=0):
    """ Lag-N cross correlation. 
    Parameters
    ----------
    lag : int, default 0
    datax, datay : pandas.Series objects of equal length

    Returns
    ----------
    crosscorr : float
    """
    return datax.corr(datay.shift(lag))

In [ ]:
len(cases.isolated)

In [ ]:
xcov = [crosscorr(cases.isolated, cases.cumLog, lag=i) for i in range(12)]
xcov

In [ ]:
# Fixing random state for reproducibility

x= cases.isolated 
y= cases.cumLog

n =len(x)
l1 = (-1/n) + (2/math.sqrt(n))

fig, [ax1, ax2] = plt.subplots(2, 1, sharex=True)
ax1.xcorr(x, y, usevlines=True, maxlags=None, normed=True, lw=2)

ax1.axhline(l1, color='blue', linestyle='dashed', lw=1)

ax1.grid(True)

ax2.acorr(x, usevlines=True, normed=True, maxlags=None, lw=2)
ax2.grid(True)


plt.show()

In [ ]:
r, p = stats.pearsonr(casesLog.dropna()['cumulative'], cases.dropna()['isolated'])
print(f"Scipy computed Pearson r: {r} and p-value: {p}")

In [ ]:
AC Scipy computed Pearson r: -0.04628281185786891 and p-value: 0.7099590018140384
        
CE Scipy computed Pearson r: 0.09147087633004868 and p-value: 0.45817151024268354
        


# Analysis of the fit results

## Data results

In [2]:
import sys
import glob

path = r'/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/results/seiir_bootstrap'
filenames = glob.glob(path + "/*.csv")
filenames

['/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/results/seiir_bootstrap/seiir_fits_boots_TO.csv',
 '/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/results/seiir_bootstrap/seiir_fits_boots_AL.csv',
 '/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/results/seiir_bootstrap/seiir_fits_boots_AM.csv',
 '/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/results/seiir_bootstrap/seiir_fits

In [3]:
lst_df = []
for f in filenames:
    print(f)
    df = pd.read_csv(f, low_memory = False, encoding="iso-8859-1")     
    #df.columns= df.columns.str.lower()
    lst_df.append(df)

/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/results/seiir_bootstrap/seiir_fits_boots_TO.csv
/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/results/seiir_bootstrap/seiir_fits_boots_AL.csv
/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/results/seiir_bootstrap/seiir_fits_boots_AM.csv
/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/results/seiir_bootstrap/seiir_fits_boots_CE.csv


In [4]:
df = pd.concat(lst_df,sort=True)
df.head()

,Unnamed: 0,beta0,beta1,beta2,delta,e0,ia0,is0,state,tcut0,tcut1,type
0,0,1.164814,0.960553,NaN,0.359780,2.675490e-06,1.642885e-06,1.310086e-09,TO,129.442472,NaN,estado
1,1,1.238291,1.071901,NaN,0.323895,3.330459e-06,1.280108e-07,2.143226e-07,TO,129.179203,NaN,estado
2,2,0.689860,1.567979,NaN,0.182781,6.235248e-06,1.906463e-06,5.989215e-06,TO,92.421808,NaN,estado
3,3,1.075234,0.832754,NaN,0.440365,1.894274e-06,5.513995e-07,1.465581e-07,TO,131.501079,NaN,estado
4,4,1.185444,0.964143,NaN,0.368267,9.302543e-07,1.954695e-06,1.544150e-07,TO,127.227360,NaN,estado


## Calculation of the confidence intervals

In [5]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.median(a), scipy.stats.sem(a)
    h = se * scipy.stats.skewnorm.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h

def day_date(t):
    t_date = []
    for days in t:
        days = np.nan_to_num(days)
        date = datetime.datetime(2020, 1, 1) + datetime.timedelta(int(days) - 1)
        t_date.append(date)

    return(t_date)

def r0(b,delta,gammaA=1/3.5,gammaS=1/4,p=0.2):

    R0a = b*delta/gammaA

    R0s = b/gammaS

    R0_geral = p*R0s + (1-p)*R0a
    
    return(R0_geral)

In [9]:
dfs = []
for uf in df.state.unique():
    for area in df.type.unique():
    
        frame = df[(df.state == uf) & (df.type == area)]


        beta0= np.array(mean_confidence_interval(frame['beta0'], confidence=0.95))

        beta0Round = np.round(mean_confidence_interval(frame['beta0'], confidence=0.95), decimals=2)

        beta1 = np.round(mean_confidence_interval(frame['beta1'], confidence=0.95), decimals=2)

        beta2 = np.round(mean_confidence_interval(frame['beta2'], confidence=0.95), decimals=2)

        delta = np.array(mean_confidence_interval(frame['delta'], confidence=0.95))

        deltaRound = np.round(mean_confidence_interval(frame['delta'], confidence=0.95), decimals=2)

        tcut0 = np.round(mean_confidence_interval(frame['tcut0'], confidence=0.95), decimals=0)

        tcut1 = np.round(mean_confidence_interval(frame['tcut1'], confidence=0.95), decimals=0)

        tcut0_date = np.array(day_date(tcut0))

        tcut1_date = np.array(day_date(tcut1))

        R0 = np.round(r0(beta0,delta),2)
    
        data = np.array([beta0, beta1,delta,tcut0,tcut0_date,tcut1,tcut1_date,R0])
        df1 = pd.DataFrame(data=data, index=['beta0','beta1','delta','tcut0','tcut0_date','tcut1','tcut1_date','R0'],columns=['median','lower','upper']).transpose()
        df1['state'] = uf
        df1['type'] = area
        dfs.append(df1)

/Users/julianeoliveira/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/julianeoliveira/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/julianeoliveira/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/julianeoliveira/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:194: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/Users/julianeoliveira/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [10]:
dfs

[          beta0 beta1     delta tcut0 tcut0_date tcut1 tcut1_date    R0 state  \
 median  1.14124  1.08  0.312494   128 2020-05-07   NaN 2019-12-31  1.91    TO   
 lower   1.07155  1.02   0.28256   125 2020-05-04   NaN 2019-12-31  1.71    TO   
 upper   1.21093  1.13  0.342427   131 2020-05-10   NaN 2019-12-31  2.13    TO   
 
           type  
 median  estado  
 lower   estado  
 upper   estado  ,
            beta0 beta1     delta tcut0 tcut0_date tcut1 tcut1_date    R0  \
 median   0.96462  0.88  0.213145   129 2020-05-08   NaN 2019-12-31  1.35   
 lower   0.882606  0.79  0.183743   125 2020-05-04   NaN 2019-12-31  1.16   
 upper    1.04663  0.97  0.242547   134 2020-05-13   NaN 2019-12-31  1.55   
 
        state     type  
 median    TO  capital  
 lower     TO  capital  
 upper     TO  capital  ,
           beta0 beta1     delta tcut0 tcut0_date tcut1 tcut1_date    R0 state  \
 median  1.38536  1.05   0.34198   129 2020-05-08   NaN 2019-12-31  2.43    TO   
 lower   1.33612  1.01

In [12]:
dfs_result = pd.concat(dfs,sort=True)

In [14]:
dfs_result.to_csv('/Users/julianeoliveira/Desktop/dropbox_15_11_2019/Artigos_preprints/COVID19/Paper_submissions/Paper2/Assessing-the-nationwide-impact-of-COVID-19-mitigation-policies-on-the-transmission-rate-of-SARS-CoV/results/ic_par.csv')

# Analises diversas

In [ ]:
overall_pearson_r = new.corr().iloc[0,1]
print(f"Pandas computed Pearson r: {overall_pearson_r}")

# Compute rolling window synchrony
f,ax=plt.subplots(figsize=(7,3))
new.rolling(window=1,center=True).median().plot(ax=ax)
ax.set(xlabel='Time',ylabel='Pearson r')
ax.set(title=f"Overall Pearson r = {np.round(overall_pearson_r,2)}");

In [ ]:
# Set window size to compute moving window synchrony.
r_window_size = 8
# Interpolate missing data.
df_interpolated = new.interpolate()
# Compute rolling window synchrony
rolling_r = df_interpolated['new_confirmed'].rolling(window=r_window_size, center=True).corr(df_interpolated['isolated'])
f,ax=plt.subplots(2,1,figsize=(14,6),sharex=True)
new.rolling(window=10,center=True).median().plot(ax=ax[0])
ax[0].set(xlabel='Frame',ylabel='Smiling Evidence')
rolling_r.plot(ax=ax[1])
ax[1].set(xlabel='Frame',ylabel='Pearson r')
plt.suptitle("Smiling data and rolling window correlation")


In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [ ]:
import functools
new.apply(functools.partial(test_stationarity))

In [ ]:
newLog.apply(functools.partial(test_stationarity))

In [ ]:
# Differencing to get the data stationary
newDiff = new.diff().dropna()
newDiff

In [ ]:
newDiff.apply(functools.partial(test_stationarity))

In [ ]:
newDiff = newDiff.apply(pd.to_numeric)

In [ ]:
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(newDiff)

In [ ]:
# Best model order (automatic selection)
model1 = model.select_order()
model1.summary()

In [ ]:
#pip install --upgrade statsmodels

#!pip install "statsmodels==0.10.2"

In [ ]:
results = model.fit(maxlags=8, ic='aic')
results.summary()

In [ ]:
results.fittedvalues

In [ ]:
# Autocorrelation function (ACF) plot of the residuals with 2=pT bounds.
results.plot_acorr()
#fig.tight_layout()
#plt.savefig('ACF_residual.pdf')


In [ ]:
from pandas import read_csv
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf

In [ ]:
plot_acf(newLog.new_confirmed)

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(new.new_confirmed, lags=50)

In [ ]:
AC, Acre; AL, Alagoas; AP, Amapá; AM, Amazonas; BA, Bahia; CE, Ceará; 

DF, Distrito Federal; ES, Espírito Santo; GO, Goiás; MA, Maranhão; MT, Mato Grosso; 

MS, Mato Grosso do Sul; MG, Minas Gerais; PA, Pará; PB, Paraíba; PR, Paraná; PE, Pernambuco;

PI, Piauí; RJ, Rio de Janeiro; RN, Rio Grande do Norte; RS, Rio Grande do Sul; 

RO, Rondônia; RR, Roraima; SC, Santa Catarina; SP, São Paulo; SE, Sergipe; TO, Tocantins.




print('AC',   3343*100000/8.944700e+05)
print('AL',     5300*100000/3.351092e+06)
print('AM',    27038*100000/4.207714e+06)
print('AP',    5655*100000/8.617730e+05)
print('BA',    12558*100000/1.493042e+07)
print('CE',    34573*100000/9.187886e+06)
print('DF',     5948*100000/3.052546e+06)
print('ES',     9520*100000/4.064052e+06)
print('GO',    2344*100000/7.116143e+06)
print('MA',    18767*100000/7.114598e+06)
print('MG',     5995*100000/2.129267e+07)
print('MS',      805*100000/2.809394e+06)
print('MT',     1271*100000/3.526220e+06)
print('PA',    22697*100000/8.690745e+06)
print('PB',     6882*100000/4.039277e+06)
print('PE',    25760*100000/9.617072e+06)
print('PI',     3258*100000/3.280697e+06)
print('PR',     2985*100000/1.151684e+07)
print('RJ',    33589*100000/1.736619e+07)
print('RN',     4598*100000/3.534165e+06)
print('RO',     2774*100000/1.796460e+06)
print('RR',     2296*100000/6.311810e+05)
print('RS',     5918*100000/1.142297e+07)
print('SC',     6458*100000/7.252502e+06)
print('SE',     4922*100000/2.319032e+06)
print('SP',    76871*100000/4.628933e+07)
print('TO',     2430*100000/1.590248e+06)           

In [ ]:
sig_uf = []
for value in mob.state_name:
    if value == 'Minas Gerais': value = 'MG'
    elif value == 'Ceará': value = 'CE'
    elif value == 'Pernambuco': value = 'PE'
    elif value == 'Mato Grosso do Sul': value = 'MS'
    elif value == 'Rio Grande do Sul': value = 'RS'
    elif value == 'Sergipe': value = 'SE'
    elif value == 'Rio Grande do Norte': value = 'RN'
    elif value == 'Acre': value = 'AC'
    elif value == 'Alagoas': value = 'AL'
    elif value == 'Bahia': value = 'BA'
    elif value == 'Rondônia': value = 'RO'
    elif value == 'Rio de Janeiro': value = 'RJ'
    elif value == 'Piauí': value = 'PI'
    elif value == 'Paraná': value = 'PR'
    elif value == 'Espírito Santo': value = 'ES'
    elif value == 'Pará': value = 'PA'
    elif value == 'Amapá': value = 'AP'
    elif value == 'Mato Grosso': value = 'MT'
    elif value == 'Paraíba': value = 'PB'
    elif value == 'Roraima': value = 'RR'
    elif value == 'Goiás': value = 'GO'
    elif value == 'São Paulo': value = 'SP'
    elif value == 'Distrito Federal': value = 'DF'
    elif value == 'Maranhão': value = 'MA'
    elif value == 'Santa Catarina': value = 'SC'
    elif value == 'Tocantins': value = 'TO'
    elif value == 'Amazonas': value = 'AM'
    sig_uf.extend([value])